In [ ]:
import ipywidgets as widgets
from IPython.display import display
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

: 

In [ ]:
def generate_key():
    key = get_random_bytes(AES.block_size)
    with open('key.bin', 'wb') as f:
        f.write(key)
    print("Key: ", key.hex())
    return key
    
key = generate_key()

In [ ]:
def generate_iv():
    iv = get_random_bytes(16)
    with open('iv.bin', 'wb') as f:
        f.write(key)
    print("Inital vector: ", iv.hex())
    return key
    
key = generate_iv()

In [ ]:
class CFB:
    def __init__(self, pl_chunk, segment_size=1, key='key.bin', iv='iv.bin'):
        with open(key, 'rb') as f:
            self.key = f.read()
        with open(iv, 'rb') as f:
            self.iv = f.read()
        self.S = self.iv
        self.segment_size = segment_size
        self.cipher = AES.new(self.key, AES.MODE_ECB)
        self.pl_chunk = pl_chunk
        self.i = 0
        self.ciphertext = b''
        self.plaintext = ''
        self.log = []
    def reset(self):
        self.i = 0
        self.ciphertext = b''
        self.plaintext = ''
        self.log.clear()
        self.S = self.iv
    def encrypt_step(self):
        if self.i == len(self.pl_chunk):
            print('End of plaintext')
            return
        S = self.S
        T = self.cipher.encrypt(S)
        k = T[:self.segment_size]
        P = self.pl_chunk[self.i].encode('utf-8')
        C = bytes([x ^ y for x, y in zip(P, k)])
        self.S = S[self.segment_size:] + C
        self.log.append({
            "Mode": "encrypt",
            "S": S.hex(),
            "Plaintext": self.pl_chunk[self.i],
            "Key": k.hex(),
            "Ciphertext": C.hex(),
            "Next_S": self.S.hex()
        })
        self.i = self.i + 1
        self.ciphertext = self.ciphertext + C
    def decrypt_step(self):
        if self.i == len(self.pl_chunk):
            print('End of ciphertext.')
            return
        S = self.S
        T = self.cipher.encrypt(S)
        k = T[:self.segment_size]
        C = bytes.fromhex(self.log[self.i]['Ciphertext'])
        P = bytes([x ^ y for x, y in zip(C, k)]).decode('utf-8')
        self.S = S[self.segment_size:] + C

        self.log.append({
            "Mode": "decrypt",
            "S": S.hex(),
            "Ciphertext": C.hex(),
            "Key": k.hex(),
            "Plaintext": P,
            "Next_S": self.S.hex()
        })

        self.plaintext = self.plaintext + P
        self.i = self.i + 1

In [ ]:
class OFB:
    def __init__(self, pl_chunk, segment_size=1, key='key.bin', iv='iv.bin'):
        with open(key, 'rb') as f:
            self.key = f.read()
        with open(iv, 'rb') as f:
            self.iv = f.read()
        
        self.segment_size = segment_size
        self.cipher = AES.new(self.key, AES.MODE_ECB)
        self.S = self.iv
        self.pl_chunk = pl_chunk
        self.i = 0
        self.ciphertext = b''
        self.plaintext = ''
        self.log = []
    def reset(self):
        self.i = 0
        self.ciphertext = b''
        self.S = self.iv
        self.log.clear()
    def encrypt_step(self):
        if self.i == len(self.pl_chunk):
            print('End of plaintext')
            return
        S = self.S
        T = self.cipher.encrypt(S)
        k = T[:self.segment_size]
        P = self.pl_chunk[self.i].encode('utf-8')
        C = bytes([x ^ y for x, y in zip(P, k)])
        self.S = self.S[self.segment_size:] + k
        self.log.append({
            "Mode": "encrypt",
            "S": S.hex(),
            "Plaintext": self.pl_chunk[self.i],
            "Key": k.hex(),
            "Ciphertext": C.hex(),
            "Next_S": self.S.hex()
        })
        self.i = self.i + 1
        self.ciphertext = self.ciphertext + C
    def decrypt_step(self):
        if self.i == len(self.pl_chunk):
            print('End of ciphertext.')
            return
        S = self.S
        T = self.cipher.encrypt(S)
        k = T[:self.segment_size]
    
        C = bytes.fromhex(self.log[self.i]['Ciphertext'])
        P = bytes([x ^ y for x, y in zip(C, k)]).decode('utf-8')
        self.S = self.S[self.segment_size:] + k

        self.log.append({
            "Mode": "decrypt",
            "S": S.hex(),
            "Ciphertext": C.hex(),
            "Key": k.hex(),
            "Plaintext": P,
            "Next_S": self.S.hex()
        })
    
        self.plaintext += P
        self.i += 1

In [ ]:
class CTR:
    def __init__(self, pl_chunk, key='key.bin', iv='iv.bin'):
        with open(key, 'rb') as f:
            self.key = f.read()
        with open(iv, 'rb') as f:
            self.iv = f.read()
        
        self.segment_size = AES.block_size
        self.cipher = AES.new(self.key, AES.MODE_ECB)
        self.Counter = int.from_bytes(self.iv, 'big')
        self.pl_chunk = pl_chunk
        self.i = 0
        self.ciphertext = b''
        self.plaintext = ''
        self.log = []
    def reset(self):
        self.i = 0
        self.ciphertext = b''
        self.Counter = int.from_bytes(self.iv, 'big')
        self.log.clear()
    def encrypt_step(self):
        Counter = self.Counter.to_bytes(AES.block_size, 'big')
        k = self.cipher.encrypt(Counter)  
        P = self.pl_chunk[self.i].encode('utf-8')
        C = bytes([x ^ y for x, y in zip(P, k)])
    
        self.log.append({
            "Mode": "encrypt",
            "Counter": self.Counter,
            "Plaintext": self.pl_chunk[self.i],
            "Key": k.hex(),
            "Ciphertext": C.hex()
        })
    
        self.Counter = (self.Counter + 1) % 2**(AES.block_size*8)
        self.i += 1
        self.ciphertext += C
    def decrypt_step(self):
        if self.i == len(self.pl_chunk):
            print("End of ciphertext")
            return
        Counter = self.Counter.to_bytes(AES.block_size, 'big')
        k = self.cipher.encrypt(Counter) 
        C = bytes.fromhex(self.log[self.i]['Ciphertext'])
        P = bytes([x ^ y for x, y in zip(C, k)]).decode('utf-8', errors='ignore')
    
        self.log.append({
            "Mode": "decrypt",
            "Counter": self.Counter,
            "Ciphertext": C.hex(),
            "Key": k.hex(),
            "Plaintext": P
        })
    
        self.Counter = (self.Counter + 1) % 2**(AES.block_size*8)
        self.i += 1
        self.plaintext += P


In [ ]:
class empty_cipher:
    def reset(self):
        pass

In [ ]:
class CipherControl:
    def __init__(self):
        self.plaintext = ''
        self.pl_chunk = []
        
        self.upload_button = widgets.FileUpload(
            accept='',
            multiple=False,
            description='Upload'
        )

        self.segment_slider = widgets.IntSlider(
            value=1, min=1, max=16, step=1,
            description='Segment'
        )

        self.split_button = widgets.Button(
            description="Split File"
        )

        self.mode_dropdown = widgets.Dropdown(
            options=['[Select Mode]', 'CFB', 'CTR', 'OFB'],
            description='Mode'
        )

        self.encrypt_button = widgets.Button(
            description="Encrypt Step"
        )

        self.file_metadata = widgets.Output()
        self.split_f_output = widgets.Output()
        self.output = widgets.Output()

        self.reset_btn = widgets.Button(description='Reset')
        self.decrypt_btn = widgets.Button(description='Decryption')
        self.prepare_de_btn = widgets.Button(description='Prepare for descryption')

        self.decrypt_output = widgets.Output()
        
        row1 = widgets.VBox([self.upload_button, self.file_metadata])
        row2 = widgets.HBox([self.segment_slider, self.split_button])
        row3 = widgets.HBox([self.mode_dropdown, self.encrypt_button])
        row4 = widgets.HBox([self.reset_btn, self.prepare_de_btn, self.decrypt_btn])
        
        ui = widgets.VBox([row1, row2, self.split_f_output, row3, self.output, row4, self.decrypt_output])
        

        display(ui)

        self.split_button.on_click(self.split_file)
        self.encrypt_button.on_click(self.encrypt_step)
        self.upload_button.observe(self.show_file, names='value')
        self.mode_dropdown.observe(self.change_mode, names='value')
        self.prepare_de_btn.on_click(self.prepare_de)
        self.decrypt_btn.on_click(self.decrypt_step)
        self.reset_btn.on_click(self.reset)

        self.cipher = empty_cipher()

    def change_mode(self, b):
        print('a')
        if self.mode_dropdown.value == 'CFB':
            self.cipher = CFB(self.pl_chunk, self.segment_slider.value)
        elif self.mode_dropdown.value == 'CTR':
            self.cipher = CTR(self.pl_chunk)
        elif self.mode_dropdown.value == 'OFB':
            self.cipher = OFB(self.pl_chunk, self.segment_slider.value)

    def show_file(self, b):
        f_name = self.upload_button.value[0]['name']
        self.plaintext = bytes(self.upload_button.value[0]['content']).decode('utf-8')
        with self.file_metadata:
            self.file_metadata.clear_output()
            print(f'File name: {f_name}')
            print(f'Content: {self.plaintext}')
    
    def split_file(self, b):
        self.pl_chunk = [self.plaintext[i:i+self.segment_slider.value] for i in range(0, len(self.plaintext), self.segment_slider.value)]
        with self.split_f_output:
            self.split_f_output.clear_output()
            for i in range(len(self.pl_chunk)):
                print(f'Segment {i}: "{self.pl_chunk[i]}"')
        
    def encrypt_step(self, b):
        if self.mode_dropdown.value == '[Select Mode]':
            print("No mode selected")
            return
        self.cipher.encrypt_step()
        with self.output:
            self.output.clear_output()
            print(f'Step: {self.cipher.i}/{len(self.cipher.pl_chunk)}')
            if self.mode_dropdown.value == 'CTR':
                print(f'Counter: {self.cipher.log[self.cipher.i-1]['Counter']}')
            if self.mode_dropdown.value != 'CTR':
                print(f'S: {self.cipher.log[self.cipher.i-1]['S']}')
            print(f'Plaintext: {self.cipher.log[self.cipher.i-1]['Plaintext']}')
            if self.mode_dropdown.value != 'CTR':
                print(f'Key: {self.cipher.log[self.cipher.i-1]['Key']}')
            print(f'Ciphertext: {self.cipher.log[self.cipher.i-1]['Ciphertext']}')
            if self.mode_dropdown.value != 'CTR':
                print(f'Next S: {self.cipher.log[self.cipher.i-1]['Next_S']}')
            print(f'> {self.cipher.ciphertext.hex()}')
    def reset(self, b):
        self.cipher.reset()
        self.output.clear_output()
        self.split_f_output.clear_output()
        self.pl_chunk.clear()
        self.cipher = empty_cipher()
        self.mode_dropdown.value = '[Select Mode]'
    def prepare_de(self, b):
        if self.mode_dropdown.value != 'CTR':
            self.cipher.S = self.cipher.iv
        else:
            self.cipher.Counter = int.from_bytes(self.cipher.iv, 'big')
        self.cipher.i = 0
    def decrypt_step(self, b):
        if self.mode_dropdown.value == '[Select Mode]':
            print("No mode selected")
            return
        self.cipher.decrypt_step()
        self.decrypt_output.clear_output()
        with self.decrypt_output:
            if self.mode_dropdown.value != 'CTR':
                print(f'S: {self.cipher.log[self.cipher.i-1]['S']}')
            print(f'Ciphertext: {self.cipher.log[self.cipher.i-1]['Ciphertext']}')
            print(f'Key: {self.cipher.log[self.cipher.i-1]['Key']}')
            print(f'Plaintext: {self.cipher.log[self.cipher.i-1]['Plaintext']}')
            if self.mode_dropdown.value != 'CTR':
                print(f'Next S: {self.cipher.log[self.cipher.i-1]['Next_S']}')
            print(f'> {self.cipher.plaintext}')

In [ ]:
test = CipherControl()